In [ ]:
import numpy as np
import pandas as pd
columns = ["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","label", "level"]
attack_type = {
    0: 'normal',
    1: 'DOS',
    2: 'Probe',
    3: 'R2L',
    4: 'U2R'
}

new_label = {
    'normal': 0,
    'apache2': 1,
    'back': 1,
    'mailbomb': 1,
    'processtable': 1,
    'snmpgetattack': 1,
    'teardrop': 1,
    'smurf': 1,
    'land': 1,
    'neptune': 1,
    'pod': 1,
    'udpstorm': 1,
    'nmap': 2,
    'ipsweep': 2,
    'portsweep': 2,
    'satan': 2,
    'mscan': 2,
    'saint': 2,
    'ftp_write': 3,
    'guess_passwd': 3,
    'snmpguess': 3,
    'imap': 3,
    'spy': 3,
    'warezclient': 3,
    'warezmaster': 3,
    'multihop': 3,
    'phf': 3,
    'imap': 3,
    'named': 3,
    'sendmail': 3,
    'xlock': 3,
    'xsnoop': 3,
    'worm': 3,
    'ps': 4,
    'buffer_overflow': 4,
    'perl': 4,
    'rootkit': 4,
    'loadmodule': 4,
    'xterm': 4,
    'sqlattack': 4,
    'httptunnel': 4
}

# Preprocessing dataset

Drop na and duplicates

In [ ]:
train = pd.read_csv("NSL-KDD/KDDTrain+.txt", header=None)
train.columns = columns
train = train.dropna()
train = train.drop_duplicates()
test = pd.read_csv("NSL-KDD/KDDTest+.txt", header=None)
test.columns = columns
test = test.dropna()
test = test.drop_duplicates()

In [ ]:
last_index_train = train.index.to_list()[-1]
df = pd.concat([train, test])

In [ ]:
df.shape

OneHotEncoder at `protocol_type`, `service` and `flag` 

In [ ]:
df.dtypes

In [ ]:
from preprocessing import encode_text_dummy
encode_text_dummy(df=df, name="protocol_type")
encode_text_dummy(df=df, name="service")
encode_text_dummy(df=df, name="flag")

In [ ]:
df

Split to train and test set

In [ ]:
train = df.iloc[:last_index_train+1]
test = df.iloc[last_index_train+1:]
X_train = train.drop(columns=['label', 'level'])
y_train = train.label.map(new_label).map(attack_type)
X_test = test.drop(columns=['label', 'level'])
y_test = test.label.map(new_label).map(attack_type)

Normalize trainset/testset

In [ ]:
from sklearn.preprocessing import MinMaxScaler
features = X_train.columns
scalser = MinMaxScaler()
scalser.fit(X_train)
X_train = scalser.transform(X_train)
X_test = scalser.transform(X_test)

SMOTE for balance dataset

In [ ]:
y_train.value_counts()

In [ ]:
from imblearn.over_sampling import SMOTE

desired_samples = {
    "U2R": 5000,
    "R2L": 5000
}

bal = SMOTE(sampling_strategy=desired_samples)

X_train, y_train = bal.fit_resample(X_train, y_train)

In [ ]:
y_train.value_counts()

# DEEP NEURAL MODEL

In [ ]:
X_train = pd.DataFrame(X_train, columns=features)
X_test = pd.DataFrame(X_test, columns=features)
y_train = pd.DataFrame(y_train, columns=['label'])
y_test = pd.DataFrame(y_test, columns=['label'])
encode_text_dummy(y_train, 'label')
encode_text_dummy(y_test, 'label')

In [ ]:
X_train.to_csv("NSL-KDD/X_train.csv")
y_train.to_csv("NSL-KDD/y_train.csv")
X_test.to_csv("NSL-KDD/X_test.csv")
y_test.to_csv("NSL-KDD/y_test.csv")

In [ ]:
X_train.head()

In [ ]:
y_train.head()

In [ ]:
import tensorflow as tf
from art.estimators.tensorflow import TensorFlowV2Estimator

layers = [
    tf.keras.layers.Dense(X_train.shape[0], activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
]
loss_object = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
dnn = tf.keras.models.Sequential(layers=layers)
dnn.compile(
    loss=loss_object,
    optimizer=optimizer,
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(),
        tf.keras.metrics.Recall(),
    ]
)
dnn.fit(
    x=X_train,
    y=y_train.values,
    epochs=5,
    batch_size=1024
)

In [ ]:
dnn.save("NSL-KDD/dnn.h5")

# SHAP and ART

load model, train and test set

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np

dnn = tf.keras.models.load_model("NSL-KDD/dnn.h5")
X_train = pd.read_csv("NSL-KDD/X_train.csv").drop(columns=['Unnamed: 0'])
y_train = pd.read_csv("NSL-KDD/y_train.csv").drop(columns=['Unnamed: 0'])
X_test = pd.read_csv("NSL-KDD/X_test.csv").drop(columns=['Unnamed: 0'])
y_test = pd.read_csv("NSL-KDD/y_test.csv").drop(columns=['Unnamed: 0'])

In [ ]:
from art.estimators.classification import TensorFlowV2Classifier
loss_object = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
dnn_model = TensorFlowV2Classifier(
    model=dnn,
    loss_object=loss_object,
    optimizer=optimizer,
    nb_classes=5,
    input_shape=X_train.shape
)

In [ ]:
from evaluation_metric import evaluate_metric
evaluate_metric(y_pred=dnn_model.predict(x=X_test), y_true=y_test.values)

In [ ]:
samples = X_test.sample(n=500)
y_samples = y_test.iloc[samples.index]

In [ ]:
from evaluation_metric import evaluate_metric
evaluate_metric(
    y_pred=dnn_model.predict(x=samples),
    y_true=y_samples
)

calculate shap values

In [ ]:
import shap
from shap_importance import shap_importance

background = pd.read_csv("NSL-KDD/samples/background.csv").drop(columns=["Unnamed: 0"])
samples = pd.read_csv("NSL-KDD/samples/samples.csv").drop(columns=["Unnamed: 0"])
y_samples = pd.read_csv("NSL-KDD/samples/y_samples.csv").drop(columns=["Unnamed: 0"])

explainer = shap.DeepExplainer(model=dnn, data=background.values)
shap_vals = explainer.shap_values(samples.values)

In [ ]:
import shap
shap.summary_plot(shap_vals, samples.columns)

accuracy plot when increase number of adversarial features

In [ ]:
import pickle
import pandas as pd
import numpy as np
import tensorflow as tf
from adversarial_n_best_worst_features import adversarial_n_best_worst_features
from shap_importance import shap_importance
from art.attacks.evasion.fast_gradient import FastGradientMethod
from art.attacks.evasion.iterative_method import BasicIterativeMethod
from art.attacks.evasion.saliency_map import SaliencyMapMethod
from art.attacks.evasion.carlini import CarliniL0Method, CarliniLInfMethod ,CarliniL2Method
from art.attacks.evasion.deepfool import DeepFool
from art.estimators.classification import TensorFlowV2Classifier

dnn = tf.keras.models.load_model("NSL-KDD/dnn.h5")
X_train = pd.read_csv("NSL-KDD/X_train.csv").drop(columns=['Unnamed: 0'])
y_train = pd.read_csv("NSL-KDD/y_train.csv").drop(columns=['Unnamed: 0'])
X_test = pd.read_csv("NSL-KDD/X_test.csv").drop(columns=['Unnamed: 0'])
y_test = pd.read_csv("NSL-KDD/y_test.csv").drop(columns=['Unnamed: 0'])


loss_object = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
dnn_model = TensorFlowV2Classifier(
    model=dnn,
    loss_object=loss_object,
    optimizer=optimizer,
    nb_classes=5,
    input_shape=X_train.shape
)


# background = pd.read_csv("NSL-KDD/samples/background.csv").drop(columns=["Unnamed: 0"])
samples = pd.read_csv("NSL-KDD/samples/samples.csv").drop(columns=["Unnamed: 0"])
y_samples = pd.read_csv("NSL-KDD/samples/y_samples.csv").drop(columns=["Unnamed: 0"])
shap_vals = pickle.load(open("NSL-KDD/samples/shap_vals_of_samples.pkl","rb"))
shap_vals = [np.array(val) for val in shap_vals]

adversarial_algs = {
    "FSGM": FastGradientMethod(estimator=dnn_model, eps=0.2),
    "BIM": BasicIterativeMethod(estimator=dnn_model, eps=0.2, max_iter=100, batch_size=32),
    "CW-L2": CarliniL2Method(classifier=dnn_model, max_iter=10),
    "JSMA": SaliencyMapMethod(classifier=dnn_model,theta=0.1,gamma=1, batch_size=1),
    "DeepFool": DeepFool(classifier=dnn_model, max_iter=100, epsilon=0.000001, nb_grads=10, batch_size=1),
}


In [ ]:
full_features = samples.columns
shap_importance_df = shap_importance(full_features, shap_val_of_sample=shap_vals)
selected_features = [f for f in shap_importance_df.loc[shap_importance_df['shap_importance']!= 0]['column_name'].values]

Generate AE sample with popular algorithm

In [ ]:
import pickle as plk
acc_list = []

for alg_name in adversarial_algs:
    print(f"{alg_name}")
    alg = adversarial_algs[alg_name]
    org_samples = samples.copy()
    adv_samples = alg.generate(x=org_samples.values) + np.random.uniform(0,10**-10, size=org_samples.shape)
    adv_samples = pd.DataFrame(adv_samples, columns=full_features)
    print(f"\t{alg_name} finish generate adversarial.")
    pd.DataFrame(np.clip(adv_samples,0,1), columns=full_features).to_csv(f"NSL-KDD/samples/attack/{alg_name}_sample.csv")

Fast Gradient Method

In [ ]:
acc_on_fsgm = adversarial_n_best_worst_features(
    model=dnn_model,
    selected_features=selected_features,
    samples=samples,
    adv_samples=pd.read_csv("NSL-KDD/samples/attack/FSGM_sample.csv"),
    y_true=y_samples
)

import matplotlib.pyplot as plt
plt.plot(
    [a+1 for a in range(len(selected_features))],
    acc_on_fsgm[0], 'red')

plt.plot(
    [a+1 for a in range(len(selected_features))],
    acc_on_fsgm[1] , 'blue')

plt.show()

Basic Interactive Method

In [ ]:
import matplotlib.pyplot as plt

acc_on_bim = adversarial_n_best_worst_features(
    model=dnn_model,
    selected_features=selected_features,
    samples=samples,
    adv_samples=pd.read_csv("NSL-KDD/samples/attack/BIM_sample.csv"),
    y_true=y_samples
)

plt.plot(
    [a+1 for a in range(len(selected_features))],
    acc_on_bim[0], 'red')

plt.plot(
    [a+1 for a in range(len(selected_features))],
    acc_on_bim[1] , 'blue')

plt.show()

Carlini L2 Method

In [ ]:
import matplotlib.pyplot as plt

acc_on_cwl2 = adversarial_n_best_worst_features(
    model=dnn_model,
    selected_features=selected_features,
    samples=samples,
    adv_samples=pd.read_csv("NSL-KDD/samples/attack/CW-L2_sample.csv"),
    y_true=y_samples
)

plt.plot(
    [a+1 for a in range(len(selected_features))],
    acc_on_cwl2[0], 'red')

plt.plot(
    [a+1 for a in range(len(selected_features))],
    acc_on_cwl2[1] , 'blue')

plt.show()

Saliency Map Method

In [ ]:
import matplotlib.pyplot as plt

acc_on_jsma = adversarial_n_best_worst_features(
    model=dnn_model,
    selected_features=selected_features,
    samples=samples,
    adv_samples=pd.read_csv("NSL-KDD/samples/attack/JSMA_sample.csv"),
    y_true=y_samples
)

plt.plot(
    [a+1 for a in range(len(selected_features))],
    acc_on_jsma[0], 'red')

plt.plot(
    [a+1 for a in range(len(selected_features))],
    acc_on_jsma[1] , 'blue')

plt.show()

DeepFool

In [ ]:
import matplotlib.pyplot as plt

acc_on_deepfool = adversarial_n_best_worst_features(
    model=dnn_model,
    selected_features=selected_features,
    samples=samples,
    adv_samples=pd.read_csv("NSL-KDD/samples/attack/DeepFool_sample.csv"),
    y_true=y_samples
)

plt.plot(
    [a+1 for a in range(len(selected_features))],
    acc_on_deepfool[0], 'red')

plt.plot(
    [a+1 for a in range(len(selected_features))],
    acc_on_deepfool[1] , 'blue')

plt.show()

# DEFENSE MECHANISM

In [105]:
import pandas as pd
import numpy as np

def generate_dataset(
    clean: pd.DataFrame,
    fsgm: pd.DataFrame,
    bim: pd.DataFrame,
    cw_l2: pd.DataFrame,
    jsma: pd.DataFrame,
    deepfool: pd.DataFrame,
    selected_features: list,
    shap_importance_feature_values: list,
    top_n_features: int = 10
):    
    n_selected_features = selected_features[:top_n_features]
    values = shap_importance_feature_values[:top_n_features]
    dfs = [
        clean[n_selected_features],
        fsgm[n_selected_features],
        bim[n_selected_features],
        cw_l2[n_selected_features],
        jsma[n_selected_features],
        deepfool[n_selected_features]
        ]
    dfs = pd.concat(dfs, axis=0, ignore_index=True) * np.array(values)
    labels = [1 for _ in range(dfs.shape[0])]
    labels[:clean.shape[0]] = [0] * clean.shape[0]
    labels = pd.DataFrame(
        {"label":labels}
    )
    
    return pd.concat([dfs, labels], axis=1).sample(frac=1, ignore_index=True)

In [130]:
import pandas as pd
# from defense.generate_dataset import generate_dataset

samples = pd.read_csv("NSL-KDD/samples/samples.csv").drop(columns=["Unnamed: 0"])
y_samples = pd.read_csv("NSL-KDD/samples/y_samples.csv").drop(columns=["Unnamed: 0"])
fsgm_samples = pd.read_csv("NSL-KDD/samples/attack/FSGM_sample.csv").drop(columns=["Unnamed: 0"])
bim_samples = pd.read_csv("NSL-KDD/samples/attack/BIM_sample.csv").drop(columns=["Unnamed: 0"])
cwl2_samples = pd.read_csv("NSL-KDD/samples/attack/CW-L2_sample.csv").drop(columns=["Unnamed: 0"])
jsma_samples = pd.read_csv("NSL-KDD/samples/attack/JSMA_sample.csv").drop(columns=["Unnamed: 0"])
deepfool_samples = pd.read_csv("NSL-KDD/samples/attack/DeepFool_sample.csv").drop(columns=["Unnamed: 0"])
shap_importance_df = pd.read_csv("NSL-KDD/samples/shap_importance.csv").drop(columns=["Unnamed: 0"])
selected_features = shap_importance_df.loc[shap_importance_df['shap_importance'] > 0]['column_name'].values
shap_importance_feature_values = shap_importance_df.loc[shap_importance_df['shap_importance'] > 0]['shap_importance'].values

defense_data = generate_dataset(
    samples,
    fsgm_samples,
    bim_samples,
    cwl2_samples,
    jsma_samples,
    deepfool_samples,
    selected_features,
    shap_importance_feature_values,
    30
)

In [131]:
defense_data

,service_auth,dst_host_srv_count,dst_host_serror_rate,srv_count,flag_S0,srv_serror_rate,count,same_srv_rate,service_http,flag_SF,...,protocol_type_tcp,dst_host_srv_rerror_rate,service_smtp,rerror_rate,service_private,service_telnet,service_ftp_data,srv_rerror_rate,hot,label
0,0.000123,117.677188,0.000097,0.197546,0.000041,0.000036,0.130385,63.191521,0.000030,57.105491,...,10.446388,0.000005,0.000004,0.000004,0.000003,0.000002,0.000002,0.000002,1.132325,1
1,0.000000,0.000000,191.241689,0.000000,80.431128,70.115578,0.000000,0.000000,0.000000,0.000000,...,0.000000,9.326567,0.000000,7.136411,5.062318,4.025770,3.760547,3.655770,3.595192,1
2,23.153524,83.738575,0.000000,0.000000,0.000000,0.000000,0.000000,46.999088,18.066351,12.957941,...,10.446393,9.326567,0.143230,5.701050,0.000000,0.431479,0.068079,3.505394,0.000000,1
3,48.537086,56.655863,152.993354,26.541898,64.344903,56.092464,33.586256,20.221296,11.939142,11.421104,...,10.446393,0.000000,1.663947,0.000000,4.049854,0.000000,0.000000,0.000000,0.000000,1
4,0.000000,169.967582,38.248338,0.000000,16.086226,14.023116,0.000000,50.553242,47.756568,45.684416,...,8.357115,1.865313,0.000000,1.427282,1.012464,0.805154,0.752109,0.731154,0.719038,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,0.000000,0.000000,191.241689,0.000000,80.431128,70.115578,0.000000,0.000000,0.000000,0.000000,...,0.000000,9.326567,0.000000,7.136411,5.062318,4.025770,3.760547,3.655770,3.595192,1
2996,0.000000,16.663488,0.000000,0.445334,0.000000,0.000000,0.273950,63.191551,0.000000,57.105519,...,10.446393,0.000000,0.000000,0.000000,0.000000,0.000000,3.760547,0.000000,0.000000,0
2997,0.000000,175.799800,0.000000,13.360015,0.000000,0.000000,0.136975,63.191551,0.000000,57.105519,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
2998,14.524358,0.000000,25.759885,0.000000,4.034734,4.579956,57.711727,0.000000,1.818373,55.892853,...,0.327255,2.744182,0.000000,1.335797,0.791149,0.010954,0.000000,0.399387,1.790483,1


In [132]:
from sklearn.model_selection import train_test_split
x_defense_train, x_defense_test, y_defense_train, y_defense_test = train_test_split(
    defense_data.drop(columns=['label']),
    defense_data['label'],
    test_size=0.3,
    stratify=defense_data['label']
)

In [133]:
y_defense_train.value_counts()

label
1    1750
0     350
Name: count, dtype: int64

In [134]:
from imblearn.over_sampling import SMOTE

balance_defense = SMOTE(sampling_strategy={0:1750})

x_defense_train, y_defense_train = balance_defense.fit_resample(x_defense_train, y_defense_train)

In [135]:
x_defense_train.head()

,service_auth,dst_host_srv_count,dst_host_serror_rate,srv_count,flag_S0,srv_serror_rate,count,same_srv_rate,service_http,flag_SF,...,diff_srv_rate,protocol_type_tcp,dst_host_srv_rerror_rate,service_smtp,rerror_rate,service_private,service_telnet,service_ftp_data,srv_rerror_rate,hot
0,1.613650e-08,212.459474,1.808741e-08,11.600947,6.851426e-09,7.234594e-11,0.136975,63.191551,59.695708,57.105519,...,2.622383e-10,10.446393,2.238005e-10,7.663275e-10,3.364019e-10,1.632212e-10,2.062095e-10,2.512434e-10,1.422024e-10,1.078558
1,0.000000e+00,27.494757,3.824834e+01,0.000000,0.000000e+00,3.225316e+01,14.135814,50.553242,0.000000,0.000000,...,2.854908e+00,10.446393,4.476752e+00,1.663947e+00,7.136411e+00,1.012464e+00,3.220616e+00,7.521095e-01,2.632154e+00,0.719038
2,1.229584e-04,135.598028,8.972861e+01,0.222130,4.075456e-05,3.552989e-05,0.136764,63.191521,0.000030,57.105491,...,7.232922e-06,10.446388,9.339733e-02,4.215607e-06,3.616206e-06,2.564791e-06,4.025768e+00,1.905421e-06,1.852407e-06,0.000002
3,0.000000e+00,212.459474,0.000000e+00,4.676005,0.000000e+00,0.000000e+00,1.506724,63.191551,59.695708,57.105519,...,0.000000e+00,10.446393,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
4,0.000000e+00,212.459474,0.000000e+00,5.344006,0.000000e+00,0.000000e+00,3.287399,63.191551,59.695708,57.105519,...,0.000000e+00,10.446393,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000


In [136]:
import tensorflow as tf
def create_cnn(n_features):
    ae_detector_dnn = tf.keras.Sequential(
        layers=[
            tf.keras.layers.Convolution1D(2048, 3, padding='same', activation='relu', input_shape=(n_features,1)),
            tf.keras.layers.MaxPooling1D(pool_size=2),
            tf.keras.layers.Convolution1D(1024, 3, padding='same', activation='relu'),
            tf.keras.layers.MaxPooling1D(pool_size=2),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(512, activation='relu'),
            tf.keras.layers.Dense(2, activation='softmax'),
        ]
    )
    
    ae_detector_dnn.compile(
        loss=tf.keras.losses.CategoricalCrossentropy(),
        optimizer=tf.keras.optimizers.Adam(),
        metrics=['accuracy']
    )
    
    return ae_detector_dnn

In [137]:
x_defense_train = np.reshape(x_defense_train, (x_defense_train.shape[0], x_defense_train.shape[1], 1))
x_defense_test = np.reshape(x_defense_test, (x_defense_test.shape[0], x_defense_test.shape[1], 1))
y_defense_train = pd.get_dummies(y_defense_train)*1
y_defense_test = pd.get_dummies(y_defense_test)*1

In [138]:
ae_detector_cnn = create_cnn(x_defense_train.shape[1])
ae_detector_cnn.fit(x_defense_train, y_defense_train, epochs=20, batch_size=32)

Epoch 1/20
110/110 [==============================] - 35s 306ms/step - loss: 1.3426 - accuracy: 0.6817
Epoch 2/20
110/110 [==============================] - 34s 312ms/step - loss: 0.4742 - accuracy: 0.7814
Epoch 3/20
110/110 [==============================] - 36s 329ms/step - loss: 0.4307 - accuracy: 0.8183
Epoch 4/20
110/110 [==============================] - 30s 272ms/step - loss: 0.3699 - accuracy: 0.8489
Epoch 5/20
110/110 [==============================] - 34s 312ms/step - loss: 0.3604 - accuracy: 0.8491
Epoch 6/20
110/110 [==============================] - 32s 293ms/step - loss: 0.3371 - accuracy: 0.8671
Epoch 7/20
110/110 [==============================] - 36s 329ms/step - loss: 0.3295 - accuracy: 0.8666
Epoch 8/20
110/110 [==============================] - 37s 337ms/step - loss: 0.3191 - accuracy: 0.8717
Epoch 9/20
110/110 [==============================] - 36s 323ms/step - loss: 0.3154 - accuracy: 0.8791
Epoch 10/20
110/110 [==============================] - 31s 282ms/step - l

In [ ]:
from evaluation_metric import evaluate_metric
evaluate_metric(y_pred=ae_detector_cnn.predict(x_defense_train), y_true=y_defense_train)

110/110 [==============================] - 1s 11ms/step


{'confusion_matrix': array([[1685,   65],
        [ 637, 1113]]),
 'accuracy_score': 0.7994285714285714,
 'precision_score': 0.7994285714285714,
 'recall_score': 0.7994285714285714,
 'f1_score': 0.7994285714285714}

In [ ]:
from evaluation_metric import evaluate_metric
evaluate_metric(y_pred=ae_detector_cnn.predict(x_defense_test), y_true=y_defense_test)

29/29 [==============================] - 0s 10ms/step


{'confusion_matrix': array([[143,   7],
        [294, 456]]),
 'accuracy_score': 0.6655555555555556,
 'precision_score': 0.6655555555555556,
 'recall_score': 0.6655555555555556,
 'f1_score': 0.6655555555555556}

evaluate model

In [ ]:
import pickle
import pandas as pd
import numpy as np
import tensorflow as tf
from adversarial_n_best_worst_features import adversarial_n_best_worst_features
from shap_importance import shap_importance
from art.attacks.evasion.fast_gradient import FastGradientMethod
from art.attacks.evasion.iterative_method import BasicIterativeMethod
from art.attacks.evasion.saliency_map import SaliencyMapMethod
from art.attacks.evasion.carlini import CarliniL0Method, CarliniLInfMethod ,CarliniL2Method
from art.attacks.evasion.deepfool import DeepFool
from art.estimators.classification import TensorFlowV2Classifier

dnn = tf.keras.models.load_model("NSL-KDD/dnn.h5")

X_test = pd.read_csv("NSL-KDD/X_test.csv").drop(columns=['Unnamed: 0'])
y_test = pd.read_csv("NSL-KDD/y_test.csv").drop(columns=['Unnamed: 0'])


loss_object = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
dnn_model = TensorFlowV2Classifier(
    model=dnn,
    loss_object=loss_object,
    optimizer=optimizer,
    nb_classes=5,
    input_shape=X_train.shape
)



def ae_detection_model(model, test_samples: pd.DataFrame, y_test_samples: pd.DataFrame, selected_features: list, n_features: int = 20):
    
    adversarial_algs = {
        "FSGM": FastGradientMethod(estimator=model, eps=0.2),
        "BIM": BasicIterativeMethod(estimator=model, eps=0.2, max_iter=100, batch_size=32),
        "CW-L2": CarliniL2Method(classifier=model, max_iter=10),
        "JSMA": SaliencyMapMethod(classifier=model,theta=0.1,gamma=1, batch_size=1),
        "DeepFool": DeepFool(classifier=model, max_iter=100, epsilon=0.000001, nb_grads=10, batch_size=1),
    }